In [1]:
import pandas as pd
import lxml
import tqdm
import requests
import time

from collections import defaultdict
from bs4 import BeautifulSoup as bs
from itertools import count
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.64 Safari/537.36'
}
max_pages = 50
movie_data = []
browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for page in count(1):
#for page in [37]:
    # request
    url = f'https://letterboxd.com/films/popular/size/small/page/{page}/'
    browser.get(url)
    time.sleep(5)
    html = browser.page_source
    soup_td = bs(html,'lxml')

    # parse data
    movies = soup_td.find_all('li', {'class': 'listitem poster-container'})

    for movie in movies:
        # general data
        data = movie.find('a').attrs['data-original-title'].split()
        title = ' '.join(data[:-2])
        year = data[-2][1:-1]
        rating = data[-1]

        # genres
        href = movie.find('a').attrs['href']
        url_detail = 'https://letterboxd.com' + href
        html_detail = requests.get(url_detail, headers=headers).content
        soup_detail = bs(html_detail, 'lxml')

        director = soup_detail.find('span', {'class': 'prettify'})
        director = None if director is None else director.text

        genres_dict = defaultdict(lambda: None)
        target_genres = ['Genre', 'Genres']
        plural_to_singular_map_genres = {'Genres': 'Genre'}
        genres_tab = soup_detail.find('div', {'id': 'tab-genres'})
        if genres_tab is not None:
            available_genres = [(idx, genre.text) for idx, genre in enumerate(genres_tab.find_all('span')) if genre.text in target_genres]
        
            genres_tab_sluglists = genres_tab.find_all('div', {'class': 'text-sluglist capitalize'})
            for idx, genre in available_genres:
                if genre in plural_to_singular_map_genres.keys():
                    genre = plural_to_singular_map_genres[genre]
                genres_dict[genre] = ', '.join(gen.text for gen in genres_tab_sluglists[idx].find_all('a'))

        # details
        details_dict = defaultdict(lambda: None)
        target_details = ['Studios', 'Countries', 'Language', 'Studio', 'Country', 'Languages']
        plural_to_singular_map = {'Studios': 'Studio', 'Countries': 'Country', 'Languages': 'Language'}
        details_tab = soup_detail.find('div', {'id': 'tab-details'})
        if details_tab is not None:
            available_details = [(idx, detail.text) for idx, detail in enumerate(details_tab.find_all('span')) if detail.text in target_details]

            details_tab_sluglists = details_tab.find_all('div', {'class': 'text-sluglist'})
            for idx, detail in available_details:
                if detail in plural_to_singular_map.keys():
                    detail = plural_to_singular_map[detail]
                details_dict[detail] = ', '.join(det.text for det in details_tab_sluglists[idx].find_all('a'))

        # cache data
        movie_data.append((title, year, rating, director, genres_dict['Genre'], details_dict['Studio'], details_dict['Country'], details_dict['Language'], href))
    if page == max_pages:
        break

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [/Users/chuy/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


WebDriverException: Message: target frame detached
  (Session info: chrome=102.0.5005.61)
Stacktrace:
0   chromedriver                        0x0000000108772d19 chromedriver + 5197081
1   chromedriver                        0x0000000108700b23 chromedriver + 4729635
2   chromedriver                        0x00000001082de16f chromedriver + 393583
3   chromedriver                        0x00000001082c8c4c chromedriver + 306252
4   chromedriver                        0x00000001082c7d5a chromedriver + 302426
5   chromedriver                        0x00000001082c825c chromedriver + 303708
6   chromedriver                        0x00000001082c81b4 chromedriver + 303540
7   chromedriver                        0x00000001082cf355 chromedriver + 332629
8   chromedriver                        0x00000001082c9556 chromedriver + 308566
9   chromedriver                        0x00000001082c9b18 chromedriver + 310040
10  chromedriver                        0x00000001082c98ca chromedriver + 309450
11  c

In [154]:
title

In [155]:
url_detail

'https://letterboxd.com/film/black-mirror-shut-up-and-dance-1/'

In [129]:
soup_detail.find('div', {'id': 'tab-genres'})

In [123]:
title

'Black Mirror: Shut Up and Dance'

# all into pandas

In [120]:
url

'https://letterboxd.com/films/popular/size/small/page/20/'

In [3]:
import pandas as pd

In [4]:
df_movies = pd.DataFrame(movie_data, columns=['movie', 'year', 'rating', 'director', 'genres', 'studios', 'country', 'languages', 'href'])

In [5]:
df_movies

,movie,year,rating,director,genres,studios,country,languages,href
0,Parasite,2019,4.58,Bong Joon-ho,"comedy, drama, thriller","Barunson E&A, CJ Entertainment",South Korea,"Korean, English, German",/film/parasite-2019/
1,Joker,2019,3.87,Todd Phillips,"crime, drama, thriller","Warner Bros. Pictures, Joint Effort, Village R...","Canada, USA",English,/film/joker-2019/
2,Knives Out,2019,4.04,Rian Johnson,"comedy, mystery, crime","Lionsgate, MRC, T-Street Productions",USA,"English, Spanish",/film/knives-out-2019/
3,Pulp Fiction,1994,4.30,Quentin Tarantino,"crime, thriller","Miramax, A Band Apart, Jersey Films",USA,"English, Spanish, French",/film/pulp-fiction/
4,Fight Club,1999,4.27,David Fincher,drama,"Regency Enterprises, Fox 2000 Pictures, Taurus...","Germany, USA",English,/film/fight-club/
...,...,...,...,...,...,...,...,...,...
3523,Quiz Show,1994,3.70,Robert Redford,"mystery, drama, history","Baltimore Pictures, Wildwood Enterprises, Holl...",USA,English,/film/quiz-show/
3524,Nineteen Eighty-Four,1984,3.39,Michael Radford,"science-fiction, thriller, drama","Atlantic Releasing Corporation, Umbrella-Rosen...",UK,English,/film/nineteen-eighty-four-1984/
3525,News from Home,1976,4.08,Chantal Akerman,documentary,"ZDF, Paradise Films, Unité Trois, Ina","Belgium, France, Germany",French,/film/news-from-home/
3526,Shadows,1959,3.63,John Cassavetes,"romance, drama",Lion International,USA,English,/film/shadows-1959/


In [8]:
df_movies.shape

(3528, 9)

In [ ]:
df_mlp.to_csv('movies.csv', index=False)